In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 10
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
queue3 = Queue(model,'Queue3',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
queue3.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))

In [5]:
# Set routing matrix with serial queue2->queue3 routing
P = model.init_routing_matrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # Serial routing: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/15045175220891375852/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.048899s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.8834  1.8834  2.0364  2.0364  0.9455  0.9518
1  Queue1   class1  5.9301  0.9415  6.2669  3.1335  0.9518  0.9480
2  Queue2   class1  0.8816  0.4671  0.8938  0.4469  0.9518  0.9444
3  Queue3   class1  5.3458  0.9335  5.6723  2.8362  0.9444  0.9444
4    Join   class1  4.1832  0.0000  2.1920  2.1920  1.9276  0.9455
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.053155s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.7139  1.7139  2.0000  2.0000  0.8569  0.8569
1  Queue1   class1  5.2236  0.8569  6.0958  3.0479  0.8569  0.8569
2  Queue2   class1  0.7360  0.4285  0.8589  0.4295  0.8569  0.8569
3  Queue3   class1  5.2236  0.8569  6.0958  3.0479  0.8569  0.8569
